# EDA Notebook

This notebook is used for exploratory data analysis

In [87]:
#imports 
import os, shutil
import pandas as pd
from sklearn.metrics import accuracy_score, recall_score

In [2]:
images_dir = 'C:/Users/User/Documents/Flatiron/Capstone/images'
annotations_dir= 'C:/Users/User/Documents/Flatiron/Capstone/annotations/trainval.txt'

In [3]:
images_list = [file for file in os.listdir(images_dir) if (file.endswith('.jpg') or
                                                          file.endswith('.JPG')  or file.endswith('.jpeg'))]

In [4]:
print('There are', len(images_list), 'train images')

There are 0 train images


In [5]:
target_vals = images_list.copy()

In [6]:
df = pd.DataFrame(target_vals, columns = ['image_name'])

In [7]:
def label_target(row):
    if row['image_name'].startswith('ABsframe'):
        return 1
    elif row['image_name'].startswith('ABbframe'):
        return 1
    elif row['image_name'].startswith('ABmframe'):
        return 1
    elif row['image_name'].startswith('billete'):
        return 0
    elif row['image_name'].startswith('DefenseKnifeAttack'):
        return 1
    elif row['image_name'].startswith('DSC'):
        return 1
    elif row['image_name'].startswith('HBbframe'):
        return 1
    elif row['image_name'].startswith('HBmframe'):
        return 1
    elif row['image_name'].startswith('HBsframe'):
        return 1
    elif row['image_name'].startswith('KnifeDefenseKrav'):
        return 1
    elif row['image_name'].startswith('knifeDefenseTechnique'):
        return 1
    elif row['image_name'].startswith('knife_'):
        return 1
    elif row['image_name'].startswith('KravMagaKnifeDefenseTechniques'):
        return 1
    elif row['image_name'].startswith('KravMagaTraining'):
        return 1
    elif row['image_name'].startswith('LBbframe'):
        return 1
    elif row['image_name'].startswith('MBbframe'):
        return 1
    elif row['image_name'].startswith('MBmframe'):
        return 1
    elif row['image_name'].startswith('MBsframe'):
        return 1
    elif row['image_name'].startswith('RealityKnife'):
        return 1
    elif row['image_name'].startswith('RusoClases'):
        return 1
    elif row['image_name'].startswith('monedero'):
        return 0
    elif row['image_name'].startswith('pistol'):
        return 1
    elif row['image_name'].startswith('sharpenKnife'):
        return 1
    elif row['image_name'].startswith('smartphone'):
        return 0
    elif row['image_name'].startswith('tarjeta'):
        return 0
    elif row['image_name'].startswith('online'):
        return 1
    elif row['image_name'].startswith('DefenseAndSurvive'):
        return 1
    elif row['image_name'].startswith('LBmframe'):
        return 1
    elif row['image_name'].startswith('LBsframe'):
        return 1

In [8]:
df['target'] = df.apply(lambda row: label_target(row), axis=1)

In [9]:
df.head()

,image_name,target


In [10]:
df.isna().sum()

image_name    0
target        0
dtype: int64

In [11]:
df['target'].value_counts(normalize=True)

Series([], Name: target, dtype: float64)

In [12]:
import time
import matplotlib.pyplot as plt
import scipy
import numpy as np
from PIL import Image
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [13]:
datagen = ImageDataGenerator()

In [15]:
image_test = load_img('C:/Users/User/Documents/Flatiron/Capstone/images/1/knife_1179.jpg').convert('RGB')

In [16]:
image_test.getpixel

<bound method Image.getpixel of <PIL.Image.Image image mode=RGB size=1280x720 at 0x290E1864310>>

In [17]:
image_test_array = img_to_array(image_test)

In [18]:
image_test_array.shape

(720, 1280, 3)

In [19]:
image_test_array = image_test_array.reshape((1,) + image_test_array.shape)
image_test_array.shape

(1, 720, 1280, 3)

## Copying from Jupyter Notebook

In [52]:
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        images_dir,target_size=(256, 256),batch_size=5002)

Found 5002 images belonging to 2 classes.


In [53]:
train_images, train_labels = next(train_generator)

In [54]:
# Explore your dataset again
m_train = train_images.shape[0]
num_px = train_images.shape[1]

In [55]:
print ("Number of training samples: " + str(m_train))
print ("train_images shape: " + str(train_images.shape))
print ("train_labels shape: " + str(train_labels.shape))

Number of training samples: 5002
train_images shape: (5002, 256, 256, 3)
train_labels shape: (5002, 2)


In [56]:
train_img = train_images.reshape(train_images.shape[0], -1)

In [57]:
print(train_img.shape)


(5002, 196608)


In [60]:
train_y = np.reshape(train_labels[:,0], (5002,1))
train_y

array([[0.],
       [1.],
       [1.],
       ...,
       [1.],
       [0.],
       [1.]], dtype=float32)

In [90]:
# attempting to create a validation set using train test split
from sklearn.model_selection import train_test_split

In [91]:
X_train, X_val, y_train, y_val = train_test_split(train_img, train_y, test_size=0.4)

In [93]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(3001, 196608)
(2001, 196608)
(3001, 1)
(2001, 1)


In [97]:
#train_images.reshape(train_images.shape[0], -1)
X_train = X_train.reshape(X_train.shape[0], -1)
X_val = X_val.reshape(X_val.shape[0], -1)
y_train = y_train.reshape(y_train.shape[0],-1)
y_val = y_val.reshape(y_val.shape[0],-1)



In [98]:
# Build a baseline fully connected model
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(20, activation='relu', input_shape=(196608,))) # 2 hidden layers
model.add(layers.Dense(7, activation='relu'))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [99]:
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

histoire = model.fit(train_img,
                    train_y,
                    epochs=50,
                    batch_size=32,
                     validation_data = (X_val,y_val))

Epoch 1/50
157/157 [==============================] - 15s 94ms/step - loss: 0.7138 - accuracy: 0.6383 - val_loss: 0.6097 - val_accuracy: 0.6747
Epoch 2/50
157/157 [==============================] - 10s 63ms/step - loss: 0.5962 - accuracy: 0.6709 - val_loss: 0.5744 - val_accuracy: 0.6877
Epoch 3/50
157/157 [==============================] - 10s 62ms/step - loss: 0.5745 - accuracy: 0.6933 - val_loss: 0.5361 - val_accuracy: 0.7226
Epoch 4/50
157/157 [==============================] - 10s 62ms/step - loss: 0.5435 - accuracy: 0.7143 - val_loss: 0.5306 - val_accuracy: 0.7121
Epoch 5/50
157/157 [==============================] - 10s 62ms/step - loss: 0.5447 - accuracy: 0.7167 - val_loss: 0.5095 - val_accuracy: 0.7531
Epoch 6/50
157/157 [==============================] - 10s 61ms/step - loss: 0.5277 - accuracy: 0.7335 - val_loss: 0.9430 - val_accuracy: 0.4708
Epoch 7/50
157/157 [==============================] - 10s 62ms/step - loss: 0.5147 - accuracy: 0.7451 - val_loss: 0.5126 - val_accuracy:

In [63]:
results_train = model.evaluate(train_img, train_y)

157/157 [==============================] - 3s 16ms/step - loss: 0.4697 - accuracy: 0.7929


In [64]:
results_train

[0.46966081857681274, 0.7928828597068787]

In [68]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(256 ,256,  3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(32, (4, 4), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer="sgd",
              metrics=['acc'])

In [71]:
history = model.fit(train_images,
                    train_y,
                    epochs=30,
                    batch_size=32,use_multiprocessing=True)

Epoch 1/30
157/157 [==============================] - 407s 3s/step - loss: 0.6139 - acc: 0.6575
Epoch 2/30
157/157 [==============================] - 377s 2s/step - loss: 0.5551 - acc: 0.7033
Epoch 3/30
157/157 [==============================] - 375s 2s/step - loss: 0.5133 - acc: 0.7393
Epoch 4/30
157/157 [==============================] - 375s 2s/step - loss: 0.4803 - acc: 0.7615
Epoch 5/30
157/157 [==============================] - 375s 2s/step - loss: 0.4308 - acc: 0.7963
Epoch 6/30
157/157 [==============================] - 379s 2s/step - loss: 0.4091 - acc: 0.8017
Epoch 7/30
157/157 [==============================] - 376s 2s/step - loss: 0.3691 - acc: 0.8323
Epoch 8/30
157/157 [==============================] - 377s 2s/step - loss: 0.3460 - acc: 0.8421
Epoch 9/30
157/157 [==============================] - 375s 2s/step - loss: 0.3238 - acc: 0.8515
Epoch 10/30
157/157 [==============================] - 383s 2s/step - loss: 0.2928 - acc: 0.8713
Epoch 11/30
157/157 [==================

In [100]:
results_train = model.evaluate(X_val, y_val)

63/63 [==============================] - 1s 16ms/step - loss: 0.3932 - accuracy: 0.8166


In [101]:
results_train

[0.39317142963409424, 0.8165916800498962]

### Test set creation

In [75]:
images_test_dir = 'C:/Users/User/Documents/Flatiron/Capstone/Test_data'


In [81]:
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        images_test_dir,
    target_size=(256, 256),
batch_size=857)

Found 857 images belonging to 2 classes.


In [82]:
test_images, test_labels = next(test_generator)

In [83]:
test_img = test_images.reshape(test_images.shape[0], -1)

In [84]:
test_y = np.reshape(test_labels[:,0], (857,1))
test_y

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],

In [86]:
test_preds = model.predict_classes(test_images)

In [89]:
#accuracy_score(y_true,y_pred)
test_accuracy = accuracy_score(test_preds,test_y)
test_recall = recall_score(test_preds,test_y)
print('accuracy score: ' + str(test_accuracy))
print('recall score: ' + str(test_recall))

accuracy score: 0.7911318553092183
recall score: 0.7918367346938775
